In [28]:
import sys
sys.path.append('/content/drive/MyDrive/multimodal sc')

In [29]:
import torch.nn as nn
import numpy as np
import torch
import pandas as pd
from GRU_pipeline import DataHolder, dicDataset, pad_collate, Pipeline
from torch.nn.utils.rnn import pad_packed_sequence

In [30]:
class GRUModel(nn.Module):
    def __init__(self, input_dim=17, hidden_dim=32, layer_dim=2, output_dim=5, dropout_prob=.1, bidirectional=False):
        super(GRUModel, self).__init__()

        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, dropout=dropout_prob, batch_first = True, bidirectional=bidirectional
        )

        if bidirectional : self.fc = nn.Linear(hidden_dim*2, output_dim)
        else: self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_packed):

        x, hidden = self.gru(x_packed)

        x,l = pad_packed_sequence(x, batch_first=True)

        out = torch.stack([x[i][l[i]-1] for i in range(x.shape[0])])

        out = self.fc(out)

        return out

In [31]:
paths={
    'path_to_embeds'            : "/content/drive/MyDrive/multimodal sc/embeds",
    'openface_spk_1_path'       : "/content/drive/MyDrive/multimodal sc/openface_speaker_1_05_by_05",
    'openface_spk_2_path'       : "/content/drive/MyDrive/multimodal sc/openface_speaker_2_05_by_05",
    'openface_spk_1_dict_path'  : "/content/drive/MyDrive/multimodal sc/openface_speaker_1_dict_utt_to_frame.txt",
    'openface_spk_2_dict_path'  : "/content/drive/MyDrive/multimodal sc/openface_speaker_2_dict_utt_to_frame.txt",
    'opensmile_spk_1_path'      : "/content/drive/MyDrive/multimodal sc/opensmile_speaker_1_05_by_05",
    'opensmile_spk_2_path'      : "/content/drive/MyDrive/multimodal sc/opensmile_speaker_2_05_by_05",
    'opensmile_spk_1_dict_path' : "/content/drive/MyDrive/multimodal sc/opensmile_speaker_1_dict_utt_to_frame.txt",
    'opensmile_spk_2_dict_path' : "/content/drive/MyDrive/multimodal sc/opensmile_speaker_2_dict_utt_to_frame.txt"
}

In [32]:
DH = DataHolder(**paths)
output = DH.make_train_test_datasets(test_size = .15, val_size = .17)
class_weights = output['class_weights'].to('cuda')
openface_1  = output['datasets']['openface_1']
openface_2  = output['datasets']['openface_2']
opensmile_1 = output['datasets']['opensmile_1']
opensmile_2 = output['datasets']['opensmile_2']

def make_results(df):
  residx = list(df.idxmax(axis=0)[:-1])
  residx.append(df.idxmin(axis=0)[-1])
  resval = list(df.max(axis=0)[:-1])
  resval.append(df.min(axis=0)[-1])
  return residx, resval

def make_a_row(model_args, DH, lr, epoch):

  args={
      'model': GRUModel(**model_args),
      'features_1':openface_1,
      'features_2':openface_2,
      'class_weights':class_weights
  }

  pipe = Pipeline(**args)
  pipe.train(epoch=epoch,lr=lr, gpu=True, early_stop=True)
  pipe.plot_losses()

  result = pd.DataFrame(pipe.hist_auf1c, columns=pipe.cats)
  result['test'] = pipe.hist_test_loss

  return make_results(result)


In [33]:
hidden_dim_list = [4, 8, 16, 32]
layer_dim_list = [2, 4, 8, 16]
dropout_prob_list = [.1, .2]

model_args_list = [{
    'input_dim':17,
    'hidden_dim':32,
    'layer_dim':2,
    'output_dim':5,
    'dropout_prob': .1,
    'bidirectional':bid }
    for bid in [True, False]]#hidden_dim in hidden_dim_list for layer_dim in layer_dim_list for dropout in dropout_prob_list]
  
loss_list = [nn.MSELoss(), nn.BCELoss()]

all_args = [{
    'model_args':model_args,
    'DH':DH,
    'lr':1e-4,
    'epoch':90
} for model_args in model_args_list for i in range(20)]# for loss in loss_list for i in range(5)]

In [ ]:
with open("/content/drive/MyDrive/multimodal sc/resultsos.txt",'w') as f:
  f.write("bidirectional,hidden_dim,layer_dim,dropout,residxSD, residxQE, residxSV, residxPR, residxHD, residexTL, resvalSD,resvalQE,resvalSV,resvalPR,resvalHD,resvalTL\n")
  for i,args in enumerate(all_args):
    bidir = args['model_args']['bidirectional']
    hidden_dim = args['model_args']['hidden_dim']
    layer_dim = args['model_args']['layer_dim']
    dropout = args['model_args']['dropout_prob']
    residx, resval = make_a_row(**args)
    residx = ','.join([str(np.round(x,3)) for x in residx])
    resval = ','.join([str(np.round(x,3)) for x in resval])
    line = f"{bidir},{hidden_dim},{layer_dim},{dropout},{residx},{resval}\n"
    f.write(line)


